In [1]:
import json
import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

import pandas as pd
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

import jiwer
from jiwer import (
    Compose,
    ToLowerCase,
    RemoveMultipleSpaces,
    Strip,
)

from IPython.display import Audio

c:\_Experiments\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def asr_metrics(hypothesis: str, reference: str):
    tr = Compose([ToLowerCase(), RemoveMultipleSpaces(), Strip()])

    ref_tr = tr(reference)
    hyp_tr = tr(hypothesis)

    out = jiwer.process_words(ref_tr, hyp_tr)
    wer = out.wer
    # S, D, I = out.substitutions, out.deletions, out.insertions

    cer = jiwer.cer(ref_tr, hyp_tr) # ?????

    return {
        "wer": wer,
        "cer": cer,
    }

In [3]:
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split="validation") 

In [4]:
Audio(dataset[116]["audio"]["array"], rate=16000)

In [8]:
metrics_log = []
with open("whisper_metric.json", "r", encoding="utf-8") as f:
    metrics = json.load(f)

metrics_log.append(metrics["large-v3_cuda"])

metrics_log

[{'total_samples': 793,
  'avg_wer': 0.4403034201331806,
  'avg_cer': 0.1582934408431861,
  'avg_time_per_audio': 0.8386273564486606,
  'total_time': 665.0314936637878}]

# PRUNE

In [4]:
# Общая функция, которая удаляет промежуточные маски и делает разреженность весов постоянной.
def finalize_model_pruning(model):
    for module in model.modules():
        if isinstance(module, (nn.Linear, nn.Conv1d)) and prune.is_pruned(module):
            prune.remove(module, 'weight')


In [5]:
# функция для подсчета метрик
def evaluate_asr(model, dataset, processor, forced_decoder_ids, device):
    results = []

    with torch.no_grad():
        for sample in tqdm(dataset):
            audio_array = sample["audio"]["array"]
            sampling_rate = sample["audio"]["sampling_rate"]
            reference = sample["transcription"]

            start_time = time.time()

            input_features = processor(
                audio_array, sampling_rate=sampling_rate, return_tensors="pt"
            ).input_features.to(device)

            predicted_ids = model.generate(
                input_features, forced_decoder_ids=forced_decoder_ids
            )[0]

            hypothesis = processor.decode(predicted_ids.cpu())

            run_time = time.time() - start_time
            metrics = asr_metrics(hypothesis, reference)
            metrics["run_time_sec"] = run_time
            results.append(metrics)

    df_results = pd.DataFrame(results)
    summary = {
        "total_samples": len(df_results),
        "avg_wer": df_results["wer"].mean(),
        "avg_cer": df_results["cer"].mean(),
        "avg_time_per_audio": df_results["run_time_sec"].mean(),
        "total_time": df_results["run_time_sec"].sum(),
    }

    return summary

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device 

'cuda'

In [6]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

forced_decoder_ids = processor.get_decoder_prompt_ids(
    language="russian", task="transcribe"
)

## Magnitude pruning

In [12]:
# Глобальное прореживание всех линейных и сверточных весов по L1-норме.
# Оставляем только (1 - pruning_rate) весов с наибольшим модулем.
def magnitude_global_pruning(model, pruning_rate):
    parameters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, (nn.Linear, nn.Conv1d)) and name != "proj_out":
            parameters_to_prune.append((module, "weight"))
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=pruning_rate,
    )

In [14]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Всего параметров в модели: {total_params/1e6:.2f}M")

Всего параметров в модели: 1543.49M


In [ ]:
# CUDA
num_iterations = 3
prune_step = 0.05

for i in range(1, num_iterations + 1):
    effective_pruning_rate = min(prune_step * i, 0.5)

    print(f"\n🔪 Итерация {i}/{num_iterations}: pruning_rate = {effective_pruning_rate}")
    magnitude_global_pruning(model, pruning_rate=effective_pruning_rate)
    
    # считаем оставшиеся параметры
    remaining_rate = (sum(mask.sum() for name, mask in model.named_buffers()) / total_params).item()
    print(f"Осталось параметров: {remaining_rate*100:.2f}%")
    
    finalize_model_pruning(model)


    model = model.to(device)
    metrics = evaluate_asr(model, dataset, processor, forced_decoder_ids, device)
    model = model.to("cpu")
    
    metrics["iteration"] = i
    metrics["remaining_params"] = remaining_rate*total_params
    metrics_log.append(metrics)

    torch.save(model.state_dict(), f"whisper_pruned_iter{i}.pt")

df_results_pruning = pd.DataFrame(metrics_log)


🔪 Итерация 1/3: pruning_rate = 0.05
Осталось параметров: 90.69%


  0%|          | 0/793 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 793/793 [08:07<00:00,  1.63it/s]



🔪 Итерация 2/3: pruning_rate = 0.1
Осталось параметров: 85.91%


100%|██████████| 793/793 [08:04<00:00,  1.64it/s]



🔪 Итерация 3/3: pruning_rate = 0.15000000000000002
Осталось параметров: 81.14%


100%|██████████| 793/793 [08:04<00:00,  1.64it/s]


In [13]:
print(df_results_pruning)

   total_samples   avg_wer   avg_cer  avg_time_per_audio  total_time  \
0            793  0.440303  0.158293            0.838627  665.031494   
1            793  0.441459  0.158367            0.612599  485.790872   
2            793  0.443707  0.162699            0.609217  483.109124   
3            793  0.440990  0.161862            0.608950  482.897073   

   iteration  remaining_params  
0        NaN               NaN  
1        1.0      1.399742e+09  
2        2.0      1.326072e+09  
3        3.0      1.252401e+09  


In [22]:
metrics["large-v3_cuda_global_magnitude_pruning_0.9"] = {
    "total_samples": metrics_log[1]["total_samples"],
    "avg_wer": metrics_log[1]["avg_wer"],
    "avg_cer": metrics_log[1]["avg_cer"],
    "avg_time_per_audio": metrics_log[1]["avg_time_per_audio"],
    "total_time": metrics_log[1]["total_time"],
}

metrics["large-v3_cuda_global_magnitude_pruning_0.85"] = {
    "total_samples": metrics_log[2]["total_samples"],
    "avg_wer": metrics_log[2]["avg_wer"],
    "avg_cer": metrics_log[2]["avg_cer"],
    "avg_time_per_audio": metrics_log[2]["avg_time_per_audio"],
    "total_time": metrics_log[2]["total_time"],
}

metrics["large-v3_cuda_global_magnitude_pruning_0.81"] = {
    "total_samples": metrics_log[3]["total_samples"],
    "avg_wer": metrics_log[3]["avg_wer"],
    "avg_cer": metrics_log[3]["avg_cer"],
    "avg_time_per_audio": metrics_log[3]["avg_time_per_audio"],
    "total_time": metrics_log[3]["total_time"],
}

In [10]:
# CPU

metrics_log = []

metrics_log.append(metrics["large-v3"])

metrics_log

[{'total_samples': 793,
  'avg_wer': 0.4403034201331806,
  'avg_cer': 0.1582934408431861,
  'avg_time_per_audio': 7.948655056021403,
  'total_time': 6303.2834594249725}]

In [15]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

num_iterations = 3
prune_step = 0.05

for i in range(1, num_iterations + 1):
    effective_pruning_rate = min(prune_step * i, 0.5)

    print(f"\n🔪 Итерация {i}/{num_iterations}: pruning_rate = {effective_pruning_rate}")
    magnitude_global_pruning(model, pruning_rate=effective_pruning_rate)
    
    # считаем оставшиеся параметры
    remaining_rate = (sum(mask.sum() for name, mask in model.named_buffers()) / total_params).item()
    print(f"Осталось параметров: {remaining_rate*100:.2f}%")
    
    finalize_model_pruning(model)

    metrics = evaluate_asr(model, dataset, processor, forced_decoder_ids, device="cpu")
    
    metrics["iteration"] = i
    metrics["remaining_params"] = remaining_rate*total_params
    metrics_log.append(metrics)

    torch.save(model.state_dict(), f"whisper_pruned_cpu_iter{i}.pt")


🔪 Итерация 1/3: pruning_rate = 0.05
Осталось параметров: 90.69%


  0%|          | 0/793 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 793/793 [1:37:56<00:00,  7.41s/it]



🔪 Итерация 2/3: pruning_rate = 0.1
Осталось параметров: 85.91%


100%|██████████| 793/793 [1:37:20<00:00,  7.36s/it]



🔪 Итерация 3/3: pruning_rate = 0.15000000000000002
Осталось параметров: 81.14%


100%|██████████| 793/793 [1:38:19<00:00,  7.44s/it]


In [19]:
metrics["large-v3_cpu_global_magnitude_pruning_0.9"] = {
    "total_samples": metrics_log[1]["total_samples"],
    "avg_wer": metrics_log[1]["avg_wer"],
    "avg_cer": metrics_log[1]["avg_cer"],
    "avg_time_per_audio": metrics_log[1]["avg_time_per_audio"],
    "total_time": metrics_log[1]["total_time"],
}

metrics["large-v3_cpu_global_magnitude_pruning_0.85"] = {
    "total_samples": metrics_log[2]["total_samples"],
    "avg_wer": metrics_log[2]["avg_wer"],
    "avg_cer": metrics_log[2]["avg_cer"],
    "avg_time_per_audio": metrics_log[2]["avg_time_per_audio"],
    "total_time": metrics_log[2]["total_time"],
}

metrics["large-v3_cpu_global_magnitude_pruning_0.81"] = {
    "total_samples": metrics_log[3]["total_samples"],
    "avg_wer": metrics_log[3]["avg_wer"],
    "avg_cer": metrics_log[3]["avg_cer"],
    "avg_time_per_audio": metrics_log[3]["avg_time_per_audio"],
    "total_time": metrics_log[3]["total_time"],
}

In [21]:
with open("whisper_metric.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=True, indent=2)

## l2 Structure pruning

In [29]:
# повторно загрузим модель
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

In [26]:
# Структурный прунинг (удаляем целые нейроны/фильтры по L2-норме).
def l2_structured_pruning(model, pruning_rate):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Linear, nn.Conv1d)) and name != "proj_out":
            prune.ln_structured(module, name="weight", amount=pruning_rate, n=2, dim=0)

In [28]:
# качество слишком сильно деградирует, поэтому показательно оставлены замеры на 5 примерах из датасета
dataset=dataset.select(range(5))

In [30]:
remaining_rate = 1.0
num_iterations = 3
prune_step = 0.05

for i in range(1, num_iterations + 1):
    effective_pruning_rate = min(prune_step * i, 0.5)

    print(f"\n🔪 Итерация {i}/{num_iterations}: pruning_rate = {effective_pruning_rate}")
    l2_structured_pruning(model, pruning_rate=effective_pruning_rate)
    # считаем оставшиеся параметры
    remaining_rate = (sum(mask.sum() for name, mask in model.named_buffers()) / total_params).item()
    print(f"Осталось параметров: {remaining_rate*100:.2f}%")
    
    finalize_model_pruning(model)


    model = model.to(device)
    metrics = evaluate_asr(model, dataset, processor, forced_decoder_ids, device)
    model = model.to("cpu")
    
    metrics["iteration"] = i
    metrics["remaining_params"] = remaining_rate*total_params
    metrics_log.append(metrics)

    torch.save(model.state_dict(), f"whisper_pruned_l2_iter{i}.pt")

df_results_pruning = pd.DataFrame(metrics_log)


🔪 Итерация 1/3: pruning_rate = 0.05
Осталось параметров: 90.69%


100%|██████████| 5/5 [00:12<00:00,  2.56s/it]



🔪 Итерация 2/3: pruning_rate = 0.1
Осталось параметров: 85.91%


100%|██████████| 5/5 [00:56<00:00, 11.39s/it]



🔪 Итерация 3/3: pruning_rate = 0.15000000000000002
Осталось параметров: 81.14%


100%|██████████| 5/5 [00:57<00:00, 11.48s/it]


In [18]:
print(df_results_pruning)

   total_samples   avg_wer     avg_cer  avg_time_per_audio  total_time  \
0            793  0.440303    0.158293            0.838627  665.031494   
1              5  0.526667    0.148921            0.775810    3.879049   
2              5  0.526667    0.148921            0.667603    3.338013   
3              5  0.526667    0.148921            0.664939    3.324694   
4              5  1.000000   29.476190            2.727848   13.639239   
5              5  1.000000  107.679610           11.313164   56.565818   
6              5  1.000000  107.679610           11.221818   56.109091   

   iteration  remaining_params  
0        NaN               NaN  
1        1.0      1.399742e+09  
2        2.0      1.326072e+09  
3        3.0      1.252401e+09  
4        1.0      1.399742e+09  
5        2.0      1.326072e+09  
6        3.0      1.252401e+09  
